In [27]:
# Initial Imports
import os
import plaid
import requests
import datetime
import json
import pandas as pd
from dotenv import load_dotenv

%matplotlib inline

In [2]:
#other functions I need to lad

def pretty_print_response(response):
    """
    This function formats a JSON file to pretty printing it in the screen.
    """
    print(json.dumps(response, sort_keys=True))

In [3]:
# load the environment
load_dotenv()

True

In [28]:
# get my keys from Plaid in order
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SECRET = os.getenv("PLAID_SECRET")
PLAID_PUBLIC_KEY = os.getenv("PLAID_PUBLIC_KEY")

In [30]:
# Create the client object
# I could never get this to work
client = plaid.Client(
    client_id = PLAID_CLIENT_ID,
    secret=PLAID_SECRET,
    public_key=PLAID_PUBLIC_KEY,
    environment="sandbox",
)

TypeError: __init__() got an unexpected keyword argument 'public_key'

In [6]:
# institution id
INSITUTION_ID = "ins_109508"

In [7]:
# Create the public token
create_response = client.Sandbox.public_token.create(
    INSITUTION_ID,
    ["transactions", "income", "assets"]
)

# Print the public token
pretty_print_response(create_response)

NameError: name 'client' is not defined

In [31]:
# Retrieve the access token
exchange_response = client.Item.public_token.exchange(create_response["public_token"])

# Print the access token
pretty_print_response(exchange_response)

NameError: name 'client' is not defined

In [9]:
# Store the access token in a Python variable
access_token = exchange_response["access_token"]

NameError: name 'exchange_response' is not defined

In [32]:
# Retrieve accounts
accounts = client.Accounts.get(access_token)

# Print accounts
pretty_print_response(accounts)

NameError: name 'client' is not defined

In [11]:
# Set the start and end date 90 days from today
start_date = "{:%Y-%m-%d}".format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = "{:%Y-%m-%d}".format(datetime.datetime.now())

# Retrieve the transactions for the last 90 days
transactions_response = client.Transactions.get(access_token, start_date, end_date)

# Print the transactions
pretty_print_response(transactions_response["transactions"][:1])

NameError: name 'client' is not defined

In [12]:
# Create a for-loop to print the categories for each transaction
for transaction in transactions_response["transactions"]:
    print(transaction["category"])

NameError: name 'transactions_response' is not defined

In [13]:
# Define an empty DataFrame and set the columns as date, name, amount, and category
transaction_df = pd.DataFrame(columns=["date", "name", "amount", "category"])

# Retrieve all the transactions
transactions = transactions_response["transactions"]

# Populate the transactions DataFrame with the transactions data
for index, transaction in enumerate(transactions):
    transaction_df.loc[index] = [transaction["date"], transaction["name"], transaction["amount"], transaction["category"][0]]

# Display sample data from the DataFrame    
transaction_df.head()

NameError: name 'transactions_response' is not defined

In [14]:
# Display data types
transaction_df.dtypes

date        object
name        object
amount      object
category    object
dtype: object

In [33]:
# Make any required data type transformation
transaction_df["amount"] = transaction_df["amount"].astype("float")
transaction_df["date"] = pd.to_datetime(transaction_df["date"], format="%Y-%m-%d")

KeyError: 'date'

In [16]:
# Set the date column as index
transaction_df = transaction_df.set_index("date")

# Display sample data
transaction_df.head()

,name,amount,category
date,,,


In [17]:
# Fetch de income data
income_response = client.Income.get(access_token)

# Print the income data
pretty_print_response(income_response)

NameError: name 'client' is not defined

In [18]:
# Determine the previous year's gross income and print the results
print(f"Last Year's income: {income_response['income']['last_year_income_before_tax']}")

NameError: name 'income_response' is not defined

In [19]:
# Determine the current monthly income and print the results
print(f"Current monthly income: {income_response['income']['income_streams'][0]['monthly_income']}")

NameError: name 'income_response' is not defined

In [20]:
# Determine the projected yearly income and print the results
print(f"Projected Year's income: {income_response['income']['projected_yearly_income_before_tax']}")

NameError: name 'income_response' is not defined

In [21]:
# Compute the total expenses per category
totals_by_category = transaction_df.groupby("category").sum()["amount"]

# Display the expenses per catefory
totals_by_category

Series([], Name: amount, dtype: float64)

In [25]:
# Create a spending categories pie chart.
totals_by_category.plot.pie(title="Total Expenses Per Category")

TypeError: Empty 'DataFrame': no numeric data to plot

In [34]:
# Create a DataFrame with the total expenses
transactions_per_month = transaction_df.reset_index()
transactions_per_month["month"] = pd.DatetimeIndex(transactions_per_month["date"]).month
transactions_per_month = transactions_per_month.groupby("month").sum()

# Display sample data
transactions_per_month

,amount
month,


In [35]:
# Create a spending per month bar chart
# Note - though 90 days of data is called, it is possible for only one month to be represented 
# This could result in as few as one bar in the final plot
transactions_per_month.plot.bar()

TypeError: Empty 'DataFrame': no numeric data to plot